In [199]:
import jax.experimental.ode as ode
import jax
import diffrax
import ticktack

In [200]:
parameters = (774.86, 0.25, 0.8, 6.44)
model = ticktack.load_presaved_model("Guttler15", production_rate_units="atoms/cm^2/s")
model.compile()

STEADY_PROD = model.equilibrate(target_C_14=707.)
STEADY_STATE = model.equilibrate(production_rate=STEADY_PROD)
PROD_COEFFS = model._production_coefficients
MATRIX = model._matrix
GROWTH = model

del model

In [201]:
@jax.tree_util.Partial
@jax.jit
def production(t, args):
    start_time, duration, phase, area = jax.numpy.array(args)
    middle = start_time + duration / 2.
    height = area / duration

    gauss = height * jax.numpy.exp(- ((t - middle) / (1. / 1.93516 * duration)) ** 16.)
    sine = 1.8803862513018528 + 0.18 * 1.8803862513018528 *\
        jax.numpy.sin(2 * jax.numpy.pi / 11 * t + phase * 2 * jax.numpy.pi / 11) 
        
    return (sine + gauss) * 3.747273140033743

In [202]:
@jax.tree_util.Partial
@jax.jit
def jax_derivative(y, t, args, /, matrix=MATRIX, production=production, prod_coeffs=PROD_COEFFS): 
    
    ans = jax.numpy.matmul(matrix, y) 
    production_rate_constant = production(t, args)
    production_term = prod_coeffs * production_rate_constant 
    return ans + production_term 

In [203]:
@jax.tree_util.Partial
@jax.jit
def diffrax_derivative(t, y, args, /, matrix=MATRIX, production=production,\
    prod_coeffs=PROD_COEFFS): 
    
    ans = jax.numpy.matmul(matrix, y) 
    production_rate_constant = production(t, args)
    production_term = prod_coeffs * production_rate_constant 
    return ans + production_term 

In [204]:
@jax.tree_util.Partial
@jax.jit
def jax_solve(y_initial, time, args, /, dydx=jax_derivative):
    states = ode.odeint(dydx, y_initial, time, args)
    return states[:, 1]

In [205]:
@jax.tree_util.Partial
@jax.jit
def diffrax_solve(y_initial, time, args, /, solver=diffrax.Bosh3(), dydx=diffrax_derivative):
    term = diffrax.ODETerm(dydx)
    saveat = diffrax.SaveAt(ts=time)
    stepsize = diffrax.PIDController(rtol=1e-5, atol=1e-5)

    t0 = time.min()
    t1 = time.max()
    dt0 = 0.001
    
    sol = diffrax.diffeqsolve(args=args, terms=term, solver=solver, t0=t0, t1=t1,\
        dt0=dt0, y0=y_initial, saveat=saveat, stepsize_controller=stepsize)
    
    return sol.ys[:, 1]

In [206]:
with open("miyake12.csv") as data:
    _ = next(data)  # String titles 
    data = jax.numpy.array([row.strip().split(" ") for row in data],\
        dtype=jax.numpy.float64)
    data = data.T

In [207]:
@jax.tree_util.Partial
@jax.jit
def bin_data(data, kernel):    
    data = data.reshape(-1, kernel.shape[0])
    data *= kernel
    data = jax.numpy.sum(data, axis=1) / jax.numpy.sum(kernel)
    return data

In [208]:
growth = jax.numpy.array([0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0], dtype=jax.numpy.float64)

In [209]:
def get_growth_kernel(oversample, /, growth=growth):
    kernel = jax.numpy.diag(growth) @ jax.numpy.ones((12, oversample // 12), dtype=jax.numpy.float64)
    kernel = kernel.flatten()
    return kernel

In [210]:
@jax.tree_util.Partial
@jax.jit
def log_likelihood(args, /, data=data, func=None, bin=bin_data, y0=STEADY_STATE, time_out=None, kernel=None):

    solution = func(y0, time_out, args)
    solution = bin(solution, kernel)
    solution = (solution - STEADY_STATE[1]) / STEADY_STATE[1]
    solution += jax.numpy.mean(data[1][:4])
    chi_squared = (solution - data[1]) ** 2 / data[2] ** 2
    return - 0.5 * jax.numpy.sum(chi_squared)

In [211]:
jac_binned = jax.jit(jax.grad(log_likelihood))
hes_binned = jax.jit(jax.jacobian(jac_binned))

In [212]:
oversample = 48
time_out = jax.numpy.linspace(data[0].min(), data[0].max() + 2, (data[0].size) * oversample)
kernel = get_growth_kernel(oversample)

In [213]:
%%timeit
log_likelihood(parameters, func=jax_solve, kernel=kernel, time_out=time_out)

The slowest run took 10.55 times longer than the fastest. This could mean that an intermediate result is being cached.
739 µs ± 734 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [214]:
%%timeit
jac_binned(parameters, func=jax_solve, kernel=kernel, time_out=time_out)

5.13 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [215]:
%%timeit
hes_binned(parameters, func=jax_solve, kernel=kernel, time_out=time_out)

324 ms ± 2.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [216]:
%%timeit
log_likelihood(parameters, func=diffrax_solve, kernel=kernel, time_out=time_out)

The slowest run took 4.07 times longer than the fastest. This could mean that an intermediate result is being cached.
512 µs ± 374 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [217]:
%%timeit
jac_binned(parameters, func=diffrax_solve, kernel=kernel, time_out=time_out)

869 ms ± 121 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [220]:
hes_binned(parameters, func=diffrax_solve, kernel=kernel, time_out=time_out)

((DeviceArray(-2.39644615, dtype=float64, weak_type=True),
  DeviceArray(-2.17127662, dtype=float64, weak_type=True),
  DeviceArray(-3.33576182e-06, dtype=float64, weak_type=True),
  DeviceArray(0.00502714, dtype=float64, weak_type=True)),
 (DeviceArray(-2.17127662, dtype=float64, weak_type=True),
  DeviceArray(-2.28539046, dtype=float64, weak_type=True),
  DeviceArray(-1.76197863e-06, dtype=float64, weak_type=True),
  DeviceArray(0.00207382, dtype=float64, weak_type=True)),
 (DeviceArray(-3.33576182e-06, dtype=float64, weak_type=True),
  DeviceArray(-1.76197863e-06, dtype=float64, weak_type=True),
  DeviceArray(-0.00619777, dtype=float64, weak_type=True),
  DeviceArray(4.58706251e-06, dtype=float64, weak_type=True)),
 (DeviceArray(0.00502714, dtype=float64, weak_type=True),
  DeviceArray(0.00207382, dtype=float64, weak_type=True),
  DeviceArray(4.58706251e-06, dtype=float64, weak_type=True),
  DeviceArray(-1.87779192e-05, dtype=float64, weak_type=True)))